# Distributional Semantics Model: Narrow Word-Window (-2,+2)

### A. Data-Loading Facilities

In [1]:
import os
os.chdir("/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03")

In [2]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from collections import defaultdict
from functools import partial
from itertools import permutations, product
punctuation = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
def load_wiki(cutoffFreq=50):
    
    print "... extracting data"
    with open('wikicorpus.txt','rb') as f:
        raw = f.readlines()
    raw = [sent[4:].split() for sent in raw if sent.startswith('<c>')] 
        # sent[4:]: get rid of initial <c>.
        # extract sentences; split sentences into word complexes.
    raw = [[map(partial(str.split, word), '|') for word in sent] for sent in raw] 
        # split word complexes into words.
    
    print "... cleaning data"
    sents = [[word[0][1].lower() for word in sent if len(word[0])>1 
              and (word[0][2].startswith('N') or word[0][2]=='.')
              and word[0][1].lower() not in stopwords
              and word[0][1] not in punctuation] for sent in raw]
        # extract lemmas => complete sents corpus .
    
    print "... building token list and vocabulary"
    tokens = [word for sent in sents for word in sent]
        # type: list of words.
    fdist = nltk.FreqDist(tokens)
    vocab = list(set(tokens))   
    
    print "... saving top %d-frequent in vocabulary" % cutoffFreq
    vocab = [word for word in vocab if fdist[word] >= cutoffFreq]
        # vocab is not returned, because the k-frequent cut latter can change it.
    sents = [[word.decode('utf-8','ignore') for word in sent if word in vocab] for sent in sents]
        # type: list of all words in the corpus, separated by periods.
        
    return sents

In [15]:
%%time
sents = load_wiki(50)

... extracting data
... cleaning data
... building token list and vocabulary
... saving top 50-frequent in vocabulary
CPU times: user 5min 5s, sys: 8.67 s, total: 5min 13s
Wall time: 5min 11s


### B1. Analyzer: Cooccurrence Matrix Based

#### A. Model

In [18]:
import numpy as np

In [19]:
# SIMILARITY MEASURES
def cosine(w2w):
    w2w_norm = w2w / np.apply_along_axis(lambda r: np.sqrt(np.dot(r,r))
                               , 1, w2w)[:,np.newaxis]
    return np.dot(w2w_norm, w2w_norm.T)
    
def ppmi(w2w):
    rowSums, colSums, totalSums = w2w.sum(axis=1), w2w.sum(axis=0), w2w.sum()
    pwi, pwj, ppmiMatrix = rowSums/totalSums, colSums/totalSums, w2w/totalSums
    ppmiMatrix /= pwi[:,np.newaxis] # * 1/pwi by row.
    ppmiMatrix /= pwj # * 1/pwj by col.
    ppmiMatrix = np.nan_to_num(np.log(ppmiMatrix)) # compute pmi.
    ppmiMatrix = np.maximum(ppmiMatrix, 0) # compute ppmi.
    return ppmiMatrix

In [28]:
class SimpleDistSem:
    
    def __init__(self, data=sents, kFrequent=50):
        self.sents = sents # a list of all words in the corpus (count=2573841).
        self.vocab = list({word for sent in sents for word in sent})
        self.wordToIndex = {word:i for i,word in enumerate(self.vocab)}
        self.indexToWord = {i:word for word,i in self.wordToIndex.iteritems()}
    
    def build_w2w_matrix(self, win_size):
        
        print "... counting words"
        cooccurrenceDict = defaultdict(int)
        for sent in self.sents:
            for i,word in enumerate(sent):
                contexts = sent[max(0,i-win_size):i] + \
                           sent[min(i+1,len(sent)):min(i+1+win_size,len(sent))]
                for context in contexts:
                    cooccurrenceDict[(self.wordToIndex[word],self.wordToIndex[context])] += 1
        
        print "... building cooccurrence matrix"
        self.w2w = np.zeros((len(self.vocab),len(self.vocab)))
        for (widx_i,widx_j),count in cooccurrenceDict.iteritems():
            self.w2w[widx_i][widx_j] = count   
    
    def build_similarity_matrix(self, similarity=ppmi):
        self.simMatrix = similarity(self.w2w)
    
    def k_most_similar(self, words, k=20):
        assert len(words)==len(filter(lambda w:1 if w in self.vocab else 0, words))
        w2sim = {}
        for word in words:
            simList = self.simMatrix[self.wordToIndex[word]]
            w2sim[word] = map(lambda idx:(self.indexToWord[idx],
                                          self.simMatrix[self.wordToIndex[word]][idx]),
                              np.argsort(simList)[::-1][1:k+1])
            # [1:k+1]: skip self-similarty.
        return w2sim

In [29]:
%%time
ds = SimpleDistSem(sents)

CPU times: user 376 ms, sys: 6.62 ms, total: 383 ms
Wall time: 380 ms


In [30]:
%%time
ds.build_w2w_matrix(win_size=2)

... counting words
... building cooccurrence matrix
CPU times: user 12 s, sys: 439 ms, total: 12.5 s
Wall time: 12.4 s


#### b. Evaluator: K-Frequent

In [31]:
%%time
ds.build_similarity_matrix(ppmi)

CPU times: user 1.66 s, sys: 546 ms, total: 2.21 s
Wall time: 2.22 s


In [32]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 8.75 ms, sys: 812 µs, total: 9.56 ms
Wall time: 9.44 ms


In [33]:
w2sim['car']

[(u'brabham', 4.7766275467196584),
 (u'bogie', 4.7055917736029063),
 (u'racing', 4.5943839798650412),
 (u'earnhardt', 4.366924146418258),
 (u'bentley', 4.3662664679992869),
 (u'brake', 4.3409486600149974),
 (u'cable', 4.3255062609917951),
 (u'clutch', 4.2657252387774101),
 (u'audi', 4.2176254439829988),
 (u'aston', 4.1303838909076473),
 (u'chassis', 4.128978409298913),
 (u'truck', 4.0516232018194867),
 (u'grip', 3.9780431663256288),
 (u'car', 3.9593991033033911),
 (u'installation', 3.9023097782248346),
 (u'motors', 3.8902269603187118),
 (u'prix', 3.8135937342977959),
 (u'tire', 3.8054304236586352),
 (u'compact', 3.7933090631262902),
 (u'crash', 3.7869508643837979)]

In [34]:
%%time
ds.build_similarity_matrix(cosine)

CPU times: user 20.8 s, sys: 272 ms, total: 21 s
Wall time: 3.54 s


In [35]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 12 ms, sys: 649 µs, total: 12.6 ms
Wall time: 11.8 ms


In [36]:
w2sim['piano']

[(u'piano', 0.99999999999999956),
 (u'cello', 0.84187590856948435),
 (u'instrument', 0.83042142568560717),
 (u'piece', 0.82429209108329438),
 (u'concerto', 0.82214317322249686),
 (u'style', 0.80750453162609459),
 (u'composition', 0.80621877549042775),
 (u'music', 0.80335035929212972),
 (u'sound', 0.80095692227873194),
 (u'sonata', 0.79813149147023077),
 (u'violin', 0.7963416282295428),
 (u'work', 0.79472264984458352),
 (u'arrangement', 0.79370701901133001),
 (u'performance', 0.79002073528385708),
 (u'career', 0.78407199429323937),
 (u'influence', 0.78106297327464669),
 (u'tone', 0.78064091165776106),
 (u'theme', 0.78052972790344377),
 (u'guitar', 0.78019642679685886),
 (u'history', 0.78010361181095722)]

#### c. Evaluator: BLESS

In [50]:
import random

In [51]:
def bless_evaluator(simMatrix=None, indexers=[None,None]):
    wordToIndex, indexToWord = indexers
    path = '/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03/BLESS_part.txt'
    with open(path,'rb') as f:
        bless = f.readlines()
    bless = [line.split('\t') for line in bless] # split into (concept, _, relation, relatum).
    crPairs = [(c.split('-')[0],r.split('-')[0],rel) for c,_,rel,r in bless]
    posPairs = [(c,r) for c,r,rel in crPairs if rel=='hyper']
    negPairs = [(c,r) for c,r,rel in crPairs if rel=='mero']
    
    return [map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), posPairs),
            map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), negPairs)]

##### PPMI

In [52]:
%%time
ds.build_similarity_matrix(ppmi)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 1.8 s, sys: 643 ms, total: 2.44 s
Wall time: 2.45 s


In [53]:
print "Examples of Evaluation on Positive Relations (PPMI): "
print random.sample(posEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in posEval])

Examples of Evaluation on Positive Relations (PPMI): 
[('bomb', 'object', 0.0), ('coat', 'clothing', 0.0), ('dolphin', 'vertebrate', 0.0), ('goat', 'creature', 3.5142631417547738), ('bull', 'creature', 0.0)]
Average PPMI:  1.41877631314


In [54]:
print "Examples of Evaluation on Negative Relations (PPMI): "
print random.sample(negEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in negEval])

Examples of Evaluation on Negative Relations (PPMI): 
[('bull', 'tail', 0.0), ('cannon', 'wheel', 1.1155047242769605), ('pub', 'light', 0.0), ('potato', 'carbohydrate', 4.0880854372575479), ('piano', 'key', 2.3543081397559291)]
Average PPMI:  1.21180615243


##### Cosine

In [55]:
%%time
ds.build_similarity_matrix(cosine)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 25.1 s, sys: 308 ms, total: 25.4 s
Wall time: 4.22 s


In [56]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(posEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in posEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('ferry', 'boat', 0.034407417781520594), ('hammer', 'tool', 0.033215358203046413), ('clarinet', 'instrument', 0.15833977866172178), ('spear', 'device', 0.004680927915039431), ('radio', 'commodity', 0.0055174583898223474)]
Average Cosine:  0.0287957750164


In [57]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(negEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in negEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('bus', 'radio', 0.015169699024820705), ('car', 'frame', 0.010182680535167509), ('grape', 'juice', 0.29588658090230935), ('car', 'radio', 0.010599621325313089), ('motorcycle', 'engine', 0.14918700503215837)]
Average Cosine:  0.0219069555561


In [13]:
sents[2]

u'anarchism'